# CS 61A Fall 2025 Notes

```sh
wget https://cs61a.org/lab/lab00/lab00.zip
unzip lab00.zip

# 用我备份的 sp22 的版本
./get.sh lab00
# sp22 需要的 python 版本
uv venv --python 3.11
```

## Textbook 1.6 高阶函数

In [9]:
def summation(n, term):
    total, k = 0, 1
    while k <= n:
        total, k = total + term(k), k + 1
    return total


def identity(x):
    return x


def cube(x):
    return x * x * x


def pi_term(x):
    return 8 / ((4 * x - 3) * (4 * x - 1))


def sum_naturals(n):
    return summation(n, identity)


def sum_cubes(n):
    return summation(n, cube)


def pi_sum(n):
    return summation(n, pi_term)


print(sum_naturals(100))
print(sum_cubes(100))
print(pi_sum(1000))

5050
25502500
3.141092653621038


In [10]:
def improve(update, close, guess=1):
    while not close(guess):
        guess = update(guess)
    return guess


def approx_eq(x, y, tolerance=1e-15):
    return abs(x - y) < tolerance

In [11]:
from math import sqrt


def golden_update(guess):
    return 1 / guess + 1


def square_close_to_successor(guess):
    return approx_eq(guess * guess, guess + 1)


def improve_test():
    phi = 1 / 2 + sqrt(5) / 2
    approx_phi = improve(golden_update, square_close_to_successor)
    assert approx_eq(phi, approx_phi), "phi differs from its approximation"


improve_test()

In [ ]:
def average(x, y):
    return (x + y) / 2


def nested_sqrt(a):
    def sqrt_update(x):
        return average(x, a / x)

    def sqrt_close(x):
        return approx_eq(x * x, a)

    return improve(sqrt_update, sqrt_close)


# 求平方根其实是构建一个面积为 a 的正方形求其边长
# 真实的平方根一定在 x 和 a/x 之间，为了接近真实值，求其平均值
print(nested_sqrt(256))

16.0


In [13]:
def newton_update(f, df):
    def update(x):
        return x - f(x) / df(x)

    return update


def find_zero(f, df):
    def near_zero(x):
        return approx_eq(f(x), 0)

    return improve(newton_update(f, df), near_zero)


def square_root_newton(a):
    def f(x):
        return x * x - a

    def df(x):
        return 2 * x

    return find_zero(f, df)


def power(x, n):
    product, k = 1, 0
    while k < n:
        product, k = product * x, k + 1
    return product


def nth_root_of_a(n, a):
    def f(x):
        return power(x, n) - a

    def df(x):
        return n * power(x, n - 1)

    return find_zero(f, df)


print(square_root_newton(64))
print(nth_root_of_a(2, 64))
print(nth_root_of_a(3, 64))
print(nth_root_of_a(6, 64))

8.0
8.0
4.0
2.0


In [18]:
def curried_pow(x):
    def h(y):
        return pow(x, y)

    return h


print(pow(2, 3))
print(curried_pow(2)(3))


def map_to_range(start, end, f):
    while start < end:
        print(f(start))
        start += 1


map_to_range(0, 10, curried_pow(2))


def curry2(f):
    def g(x):
        def h(y):
            return f(x, y)

        return h

    return g


def uncurry2(g):
    def f(x, y):
        return g(x)(y)

    return f


pow_curried = curry2(pow)
print(pow_curried(2)(5))
print(uncurry2(pow_curried)(2, 5))


8
8
1
2
4
8
16
32
64
128
256
512
32
32


In [ ]:
def trace(fn):
    def wrapped(x):
        print("-> ", fn, "(", x, ")")
        return fn(x)

    return wrapped


# 等价于 triple = trace(triple)
@trace
def triple(x):
    return 3 * x


triple(12)

->  <function triple at 0x71e94af64a90> ( 12 )


36

## Textbook 1.7 递归函数

In [2]:
def sum_digits(n):
    if n < 10:
        return n
    all_but_last, last = n // 10, n % 10
    return sum_digits(all_but_last) + last


print(sum_digits(9))
print(sum_digits(18117))
print(sum_digits(9437184))

9
18
36


In [3]:
def fact_iter(n):
    total, k = 1, 1
    while k <= n:
        total, k = total * k, k + 1
    return total


def fact(n):
    if n == 1:
        return 1
    return n * fact(n - 1)


print(fact_iter(4))
print(fact(4))

24
24


In [4]:
def is_even(n):
    if n == 0:
        return True
    return is_odd(n - 1)


def is_odd(n):
    if n == 0:
        return False
    return is_even(n - 1)


def is_even2(n):
    if n == 0:
        return True
    else:
        if (n - 1) == 0:
            return False
        return is_even((n - 1) - 1)


print(is_even(4))
print(is_even2(4))

True
True


In [6]:
def cascade(n):
    """打印 n 的前缀级联"""
    if n < 10:
        print(n)
    else:
        # print(n)
        cascade(n // 10)
        print(n)


cascade(2013)

2
20
201
2013


In [7]:
def play_alice(n):
    if n == 0:
        print("Bob wins!")
    else:
        play_bob(n - 1)


def play_bob(n):
    if n == 0:
        print("Alice wins!")
    elif is_even(n):
        play_alice(n - 2)
    else:
        play_alice(n - 1)


play_alice(20)

Bob wins!


In [8]:
def fib(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib(n - 2) + fib(n - 1)


fib(6)

5

In [ ]:
def count_partitions(n, m):
    """计算使用最大数 m 的整数分割 n 的方法数"""
    if n == 0:
        return 1
    elif n < 0:
        return 0
    elif m == 0:
        return 0
    else:
        return count_partitions(n - m, m) + count_partitions(n, m - 1)


count_partitions(6, 4)

9

## Disc03 递归

In [4]:
def multiply(m, n):
    """Takes two positive integers and returns their product using recursion.
    >>> multiply(5, 3)
    15
    """
    if n == 1:
        return m
    return m + multiply(m, n - 1)


print(multiply(5, 3))
print(multiply(9, 8))
print(multiply(19, 18))

15
72
342


In [5]:
def skip_mul(n):
    """Return the product of n * (n - 2) * (n - 4) * ...

    >>> skip_mul(5) # 5 * 3 * 1
    15
    >>> skip_mul(8) # 8 * 6 * 4 * 2
    384
    """
    if n == 1:
        return 1
    if n == 2:
        return 2
    return n * skip_mul(n - 2)


print(skip_mul(5))
print(skip_mul(8))

15
384


In [8]:
def hailstone(n):
    """Print out the hailstone sequence starting at n, and return the number of elements in the sequence.
    >>> a = hailstone(10)
    10
    5
    16
    8
    4
    2
    1
    >>> a
    7
    >>> b = hailstone(1)
    1
    >>> b
    1
    """
    print(n)
    if n == 1:
        return 1
    if n % 2 == 0:
        return 1 + hailstone(n // 2)
    return 1 + hailstone(3 * n + 1)


hailstone(10)

10
5
16
8
4
2
1


7

In [10]:
def merge(n1, n2):
    """Merges two numbers by digit in decreasing order
    >>> merge(31, 42)
    4321
    >>> merge(21, 0)
    21
    >>> merge (21, 31)
    3211
    """
    if n1 == 0:
        return n2
    if n2 == 0:
        return n1
    if n1 % 10 < n2 % 10:
        return merge(n1 // 10, n2) * 10 + n1 % 10
    return merge(n1, n2 // 10) * 10 + n2 % 10


merge(31, 42)

4321

In [12]:
def is_prime(n):
    """Returns True if n is a prime number and False otherwise.

    >>> is_prime(2)
    True
    >>> is_prime(16)
    False
    >>> is_prime(521)
    True
    """

    def helper(i):
        if i * i > n:
            return True
        if n % i == 0:
            return False
        return helper(i + 1)

    return helper(2)


is_prime(521)

True